In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import binance
import yfinance as yf
import seaborn as sns
from binance.client import Client
import datetime as dt
import os
import json
import requests
from datetime import datetime, timedelta
import pandas as pd

In [2]:
# client configuration
api_key = os.environ["BINANCE_API_KEY"]
api_secret = os.environ["BINANCE_API_SECRET"]  ## put your secrets in enviroment variables
client = Client(api_key, api_secret)

 ## Binance official api

#### Data information: 
    1499040000000,      // Open time
    "0.01634790",       // Open
    "0.80000000",       // High
    "0.01575800",       // Low
    "0.01577100",       // Close
    "148976.11427815",  // Volume (in base asset ETH)
    1499644799999,      // Close time
    "2434.19055334",    // Quote asset volume (in quote asset USDT)
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "17928899.62484339" // Ignore

### Getting 1 minutes chart for short term investigation

In [3]:
start_time = datetime.strptime('2017-09-01 00:00:00', '%Y-%m-%d %H:%M:%S') ## first timestamp available on Binance

In [4]:
list_of_millisecs =[]
now = datetime.now()

In [5]:
while start_time < now:
    millisec = start_time.timestamp() * 1000
    list_of_millisecs.append(str(int(millisec)))
    start_time = start_time + timedelta(hours=12)

In [6]:
df = pd.DataFrame()

In [7]:
for i in range(0, len(list_of_millisecs)):
    try:
        url = 'https://api.binance.com/api/v3/klines'
        symbol = 'BTCUSDT'
        interval = '1m'
        start = list_of_millisecs[i]
        if i+1 == len(list_of_millisecs):
            end = str(int(datetime.now().timestamp() * 1000))
        else:
            end = list_of_millisecs[i+1]

        par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end, 'limit': 1000}
        data = pd.DataFrame(json.loads(requests.get(url, params= par).text))

        #format columns name
        data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
        data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
        data=data.astype(float)
        
    except:
        pass
    
    
    df = pd.concat([df, data])

In [8]:
df.head()

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2017-09-01 00:00:00,1.504217e+12,4724.71,4724.71,4724.71,4724.71,0.000000,1.504217e+12,0.000000e+00,0.0,0.000000,0.000000e+00,0.0
2017-09-01 00:01:00,1.504217e+12,4724.71,4724.71,4724.71,4724.71,0.000000,1.504217e+12,0.000000e+00,0.0,0.000000,0.000000e+00,0.0
2017-09-01 00:02:00,1.504217e+12,4685.03,4685.03,4685.03,4685.03,0.078030,1.504217e+12,3.655729e+02,1.0,0.000000,0.000000e+00,0.0
2017-09-01 00:03:00,1.504217e+12,4685.00,4685.00,4685.00,4685.00,0.118337,1.504217e+12,5.544088e+02,2.0,0.000000,0.000000e+00,0.0
2017-09-01 00:04:00,1.504217e+12,4694.99,4694.99,4694.99,4694.99,0.073242,1.504217e+12,3.438705e+02,1.0,0.073242,3.438705e+02,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-08 12:37:00,1.678275e+12,22073.16,22079.71,22071.00,22076.38,174.722170,1.678275e+12,3.857036e+06,4513.0,88.989250,1.964473e+06,0.0
2023-03-08 12:38:00,1.678275e+12,22076.38,22083.01,22075.23,22077.72,86.202820,1.678276e+12,1.903293e+06,3478.0,44.741190,9.878619e+05,0.0
2023-03-08 12:39:00,1.678276e+12,22077.72,22087.02,22077.68,22084.44,99.245820,1.678276e+12,2.191631e+06,3341.0,56.634570,1.250655e+06,0.0
2023-03-08 12:40:00,1.678276e+12,22084.00,22091.75,22082.65,22089.09,123.475670,1.678276e+12,2.727240e+06,3616.0,70.814270,1.564114e+06,0.0


In [9]:
df.drop_duplicates(keep="first", inplace=True)

In [12]:
df.to_csv("src/minute/btc/btc_1_min_df.csv", index=False)

In [13]:
df.tail()

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2023-03-08 12:37:00,1.678275e+12,22073.16,22079.71,22071.00,22076.38,174.72217,1.678275e+12,3.857036e+06,4513.0,88.98925,1.964473e+06,0.0
2023-03-08 12:38:00,1.678275e+12,22076.38,22083.01,22075.23,22077.72,86.20282,1.678276e+12,1.903293e+06,3478.0,44.74119,9.878619e+05,0.0
2023-03-08 12:39:00,1.678276e+12,22077.72,22087.02,22077.68,22084.44,99.24582,1.678276e+12,2.191631e+06,3341.0,56.63457,1.250655e+06,0.0
2023-03-08 12:40:00,1.678276e+12,22084.00,22091.75,22082.65,22089.09,123.47567,1.678276e+12,2.727240e+06,3616.0,70.81427,1.564114e+06,0.0
2023-03-08 12:41:00,1.678276e+12,22089.75,22094.36,22089.09,22093.01,36.77758,1.678276e+12,8.124908e+05,1186.0,19.87581,4.390933e+05,0.0


### Getting daily chart for short-mid term investigation

In [14]:
df = pd.DataFrame()

In [15]:
for i in range(0, len(list_of_millisecs)):
    try:
        url = 'https://api.binance.com/api/v3/klines'
        symbol = 'BTCUSDT'
        interval = '1d'
        start = list_of_millisecs[i]
        if i+1 == len(list_of_millisecs):
            end = str(int(datetime.now().timestamp() * 1000))
        else:
            end = list_of_millisecs[i+1]

        par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end, 'limit': 1000}
        data = pd.DataFrame(json.loads(requests.get(url, params= par).text))

        #format columns name
        data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
        data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
        data=data.astype(float)
        
    except:
        pass
    
    
    df = pd.concat([df, data])

In [16]:
df.tail()

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2023-03-04 01:00:00,1.677888e+12,22354.34,22410.00,22157.08,22346.57,121257.38132,1.677974e+12,2.706423e+09,4169260.0,60043.33153,1.340205e+09,0.0
2023-03-05 01:00:00,1.677974e+12,22346.57,22662.09,22189.22,22430.24,154841.75786,1.678061e+12,3.473011e+09,4835978.0,77394.35765,1.735989e+09,0.0
2023-03-06 01:00:00,1.678061e+12,22430.24,22602.19,22258.00,22410.00,203751.82957,1.678147e+12,4.569102e+09,6471278.0,102110.26304,2.289889e+09,0.0
2023-03-07 01:00:00,1.678147e+12,22409.41,22557.91,21927.00,22197.96,292519.80912,1.678234e+12,6.517595e+09,7813394.0,145498.64219,3.242137e+09,0.0
2023-03-08 01:00:00,1.678234e+12,22198.56,22287.00,21840.13,22083.10,119167.30507,1.678320e+12,2.630113e+09,3357445.0,59505.45077,1.313373e+09,0.0


In [19]:
df.to_csv("src/daily/btc/btc_daily_df.csv", index=False)

### Getting weekly chart for mid-long term investigation

In [20]:
df = pd.DataFrame()

In [21]:
for i in range(0, len(list_of_millisecs)):
    try:
        url = 'https://api.binance.com/api/v3/klines'
        symbol = 'BTCUSDT'
        interval = '1w'
        start = list_of_millisecs[i]
        if i+1 == len(list_of_millisecs):
            end = str(int(datetime.now().timestamp() * 1000))
        else:
            end = list_of_millisecs[i+1]

        par = {'symbol': symbol, 'interval': interval, 'startTime': start, 'endTime': end, 'limit': 1000}
        data = pd.DataFrame(json.loads(requests.get(url, params= par).text))

        #format columns name
        data.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
        data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.datetime]
        data=data.astype(float)
        
    except:
        pass
    
    
    df = pd.concat([df, data])

In [22]:
df.tail()

,datetime,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
2023-02-06 01:00:00,1.675642e+12,22932.91,23452.00,21451.00,21783.54,1.976379e+06,1.676246e+12,4.435382e+10,48694515.0,9.814532e+05,2.202854e+10,0.0
2023-02-13 01:00:00,1.676246e+12,21782.37,25250.00,21351.07,24271.76,2.497565e+06,1.676851e+12,5.873952e+10,58393471.0,1.250658e+06,2.941542e+10,0.0
2023-02-20 01:00:00,1.676851e+12,24272.51,25250.00,22722.00,23554.85,2.237984e+06,1.677456e+12,5.375002e+10,73475940.0,1.113474e+06,2.674707e+10,0.0
2023-02-27 01:00:00,1.677456e+12,23554.85,24000.00,21971.13,22430.24,1.698503e+06,1.678061e+12,3.922468e+10,55369117.0,8.443098e+05,1.950010e+10,0.0
2023-03-06 01:00:00,1.678061e+12,22430.24,22602.19,21840.13,22036.64,6.219140e+05,1.678666e+12,1.385967e+10,17823750.0,3.103918e+05,6.917711e+09,0.0


In [26]:
df.to_csv("src/weekly/btc/btc_weekly_df.csv", index=False)